In [1]:
import pandas as pd 
import numpy as np 

df_train = pd.read_csv('clean_data/train.csv')
df_test = pd.read_csv('clean_data/test.csv')

target_cols = ['EC1', 'EC2']
num_cols = ['BertzCT', 'Chi1', 'Chi1n', 'Chi1v', 'Chi2n', 'Chi2v', 'Chi3v',
            'Chi4n', 'EState_VSA1', 'EState_VSA2', 'ExactMolWt', 'FpDensityMorgan1',
            'FpDensityMorgan2', 'FpDensityMorgan3', 'HallKierAlpha',
            'HeavyAtomMolWt', 'Kappa3', 'MaxAbsEStateIndex', 'MinEStateIndex',
            'NumHeteroatoms', 'PEOE_VSA10', 'PEOE_VSA14', 'PEOE_VSA6', 'PEOE_VSA7',
            'PEOE_VSA8', 'SMR_VSA10', 'SMR_VSA5', 'SlogP_VSA3', 'VSA_EState9',
            'fr_COO', 'fr_COO2']
binary_cols = ['EC3', 'EC4', 'EC5', 'EC6']

x_train = df_train[num_cols].to_numpy()
y_train = df_train[target_cols].to_numpy()

x_test = df_test[num_cols].to_numpy()

In [2]:
from sklearn.model_selection import train_test_split

x_train, x_cv, y_train, y_cv = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

In [3]:
import tensorflow as tf
from sklearn.metrics import roc_auc_score
import optuna

def objective_ec1(trial):
    # Define the hyperparameters to be tuned
    num_layers = trial.suggest_int('num_layers', 1, 3)
    num_units = trial.suggest_int('num_units', 16, 128)
    regularisation = trial.suggest_float('regularisation', 0.0, 0.1)
    epochs = trial.suggest_int('epochs', 5, 20)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64])

    # Define the model architecture
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(num_units, activation='relu', input_shape=(x_train.shape[1],)))
    for _ in range(num_layers):
        model.add(tf.keras.layers.Dense(num_units, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(regularisation)))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

    # Compile the model with AUC-ROC as the metric
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=[tf.keras.metrics.AUC(curve='ROC')])

    # Train the model
    model.fit(x_train, y_train[:, 0], epochs=epochs, batch_size=batch_size, validation_data=(x_cv, y_cv[:, 0]), verbose=0)

    # Calculate the AUC-ROC score on the validation set
    y_pred = model.predict(x_cv)
    auc_roc = roc_auc_score(y_cv[:, 0], y_pred)

    return auc_roc

def objective_ec2(trial):
    # Define the hyperparameters to be tuned
    num_layers = trial.suggest_int('num_layers', 1, 3)
    num_units = trial.suggest_int('num_units', 16, 128)
    regularisation = trial.suggest_float('regularisation', 0.0, 0.1)
    epochs = trial.suggest_int('epochs', 5, 20)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64])

    # Define the model architecture
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(num_units, activation='relu', input_shape=(x_train.shape[1],)))
    for _ in range(num_layers):
        model.add(tf.keras.layers.Dense(num_units, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(regularisation)))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

    # Compile the model with AUC-ROC as the metric
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=[tf.keras.metrics.AUC(curve='ROC')])

    # Train the model
    model.fit(x_train, y_train[:, 1], epochs=epochs, batch_size=batch_size, validation_data=(x_cv, y_cv[:, 1]), verbose=0)

    # Calculate the AUC-ROC score on the validation set
    y_pred = model.predict(x_cv)
    auc_roc = roc_auc_score(y_cv[:, 1], y_pred)

    return auc_roc


In [4]:
# Create Optuna study for model_ec1
study_ec1 = optuna.create_study(direction='maximize')
study_ec1.optimize(objective_ec1, n_trials=50)

# Get the best hyperparameters and best AUC-ROC score for model_ec1
best_params_ec1 = study_ec1.best_params
best_auc_roc_ec1 = study_ec1.best_value

# Define the best model architecture for model_ec1
best_model_ec1 = tf.keras.Sequential()
best_model_ec1.add(tf.keras.layers.Dense(best_params_ec1['num_units'], activation='relu', input_shape=(x_train.shape[1],)))
for _ in range(best_params_ec1['num_layers']):
    best_model_ec1.add(tf.keras.layers.Dense(best_params_ec1['num_units'], activation='relu', kernel_regularizer=tf.keras.regularizers.l2(best_params_ec1['regularisation'])))
best_model_ec1.add(tf.keras.layers.Dense(1, activation='sigmoid'))

# Compile the best model for model_ec1
best_model_ec1.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=[tf.keras.metrics.AUC(curve='ROC')])

# Train the best model for model_ec1
best_model_ec1.fit(x_train, y_train[:, 0], epochs=best_params_ec1['epochs'], batch_size=best_params_ec1['batch_size'], validation_data=(x_cv, y_cv[:, 0]), verbose=0)

# Get the predicted probabilities on the test set for model_ec1
cv_y_pred_prob_1 = best_model_ec1.predict(x_cv)
train_y_pred_prob_1 = best_model_ec1.predict(x_train)

# Calculate AUC-ROC score on the test set for model_ec1
train_auc_roc_1 = roc_auc_score(y_train[:, 0], train_y_pred_prob_1)
cv_auc_roc_1 = roc_auc_score(y_cv[:, 0], cv_y_pred_prob_1)


# Create Optuna study for model_ec2
study_ec2 = optuna.create_study(direction='maximize')
study_ec2.optimize(objective_ec2, n_trials=50)

# Get the best hyperparameters and best AUC-ROC score for model_ec2
best_params_ec2 = study_ec2.best_params
best_auc_roc_ec2 = study_ec2.best_value

# Define the best model architecture for model_ec2
best_model_ec2 = tf.keras.Sequential()
best_model_ec2.add(tf.keras.layers.Dense(best_params_ec2['num_units'], activation='relu', input_shape=(x_train.shape[1],)))
for _ in range(best_params_ec2['num_layers']):
    best_model_ec2.add(tf.keras.layers.Dense(best_params_ec2['num_units'], activation='relu', kernel_regularizer=tf.keras.regularizers.l2(best_params_ec2['regularisation'])))
best_model_ec2.add(tf.keras.layers.Dense(1, activation='sigmoid'))

# Compile the best model for model_ec2
best_model_ec2.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=[tf.keras.metrics.AUC(curve='ROC')])

# Train the best model for model_ec2
best_model_ec2.fit(x_train, y_train[:, 1], epochs=best_params_ec2['epochs'], batch_size=best_params_ec2['batch_size'], validation_data=(x_cv, y_cv[:, 1]), verbose=0)

# Get the predicted probabilities on the test set for model_ec2
cv_y_pred_prob_2 = best_model_ec2.predict(x_cv)
train_y_pred_prob_2 = best_model_ec2.predict(x_train)

# Calculate AUC-ROC score on the test set for model_ec2
train_auc_roc_2 = roc_auc_score(y_train[:, 1], train_y_pred_prob_2)
cv_auc_roc_2 = roc_auc_score(y_cv[:, 1], cv_y_pred_prob_2)


print('\n', '='*150, '\n')

print("Train AUC-ROC score 1:", train_auc_roc_1)
print("Train AUC-ROC score 2:", train_auc_roc_2)

print("CV AUC-ROC score 1:", cv_auc_roc_1)
print("CV AUC-ROC score 2:", cv_auc_roc_2)

print("\nAvg AUC-ROC score: ", (cv_auc_roc_1 + cv_auc_roc_2)/2)


[I 2023-07-10 14:27:06,874] A new study created in memory with name: no-name-5973ce17-f41b-4eab-a73d-7a681bf62197


93/93 [==============================] - 0s 2ms/step


[I 2023-07-10 14:27:15,493] Trial 0 finished with value: 0.705157721048127 and parameters: {'num_layers': 2, 'num_units': 70, 'regularisation': 0.08792828892304248, 'epochs': 8, 'batch_size': 32}. Best is trial 0 with value: 0.705157721048127.


93/93 [==============================] - 0s 1ms/step


[I 2023-07-10 14:27:33,873] Trial 1 finished with value: 0.6999756196918823 and parameters: {'num_layers': 1, 'num_units': 19, 'regularisation': 0.026257927246995463, 'epochs': 16, 'batch_size': 16}. Best is trial 0 with value: 0.705157721048127.


...

93/93 [==============================] - 0s 1ms/step


[I 2023-07-10 14:40:32,301] Trial 48 finished with value: 0.582918867370892 and parameters: {'num_layers': 1, 'num_units': 73, 'regularisation': 0.0005064917731468646, 'epochs': 6, 'batch_size': 16}. Best is trial 44 with value: 0.5947014377934272.


93/93 [==============================] - 0s 1ms/step


[I 2023-07-10 14:40:39,147] Trial 49 finished with value: 0.5827167693661972 and parameters: {'num_layers': 2, 'num_units': 58, 'regularisation': 0.0049495648841783955, 'epochs': 5, 'batch_size': 16}. Best is trial 44 with value: 0.5947014377934272.


371/371 [==============================] - 0s 1ms/step


Train AUC-ROC score 1: 0.6996886791584781
Train AUC-ROC score 2: 0.6026649061206346
CV AUC-ROC score 1: 0.6942426468167754
CV AUC-ROC score 2: 0.5863134536384976

Avg AUC-ROC score:  0.6402780502276365


In [5]:
y_pred_1, y_pred_2 = best_model_ec1.predict(x_test), best_model_ec2.predict(x_test)

310/310 [==============================] - 0s 950us/step


In [6]:
# Flattening the multidimensional arrays
y_pred_1_flat = y_pred_1.flatten()
y_pred_2_flat = y_pred_2.flatten()

# Creating the DataFrames
df_y_pred_1 = pd.DataFrame({'EC1': y_pred_1_flat})
df_y_pred_2 = pd.DataFrame({'EC2': y_pred_2_flat})

# Rest of the code remains the same
ids = df_test['id']
df_ids = pd.DataFrame({'id': ids})

result = pd.concat([df_ids, df_y_pred_1, df_y_pred_2], axis=1)

result.to_csv('submissions/submission_2_neural_net_3.csv', index=False)
